# How to read a TOPS SLC product with xsar ?

the TOPS (IW and EW) SLC products are distributed by ESA in .SAFE directories containing measurement and annotations per subswath and polarizations.


For instance in a IW SLC product you will find 6 .tiff files giving the Digital Number of each polarizations/subswath:
 - s1a-iw1-slc-vh-20210106t164941-20210106t165006-036020-043871-001.tiff  
 - s1a-iw2-slc-vh-20210106t164942-20210106t165007-036020-043871-002.tiff  
 - s1a-iw3-slc-vh-20210106t164940-20210106t165008-036020-043871-003.tiff
 - s1a-iw1-slc-vv-20210106t164941-20210106t165006-036020-043871-004.tiff  
 - s1a-iw2-slc-vv-20210106t164942-20210106t165007-036020-043871-005.tiff
 - s1a-iw3-slc-vv-20210106t164940-20210106t165008-036020-043871-006.tiff

## to open a specific subswath

In [ ]:
import xsar
ff_safe = '/home/datawork-cersat-public/project/mpc-sentinel1/data/esa/sentinel-1b/L1/IW/S1B_IW_SLC__1S/2018/001/S1B_IW_SLC__1SDV_20180101T033427_20180101T033454_008972_010031_BC89.SAFE'
ff_safe = 'S1A_IW_SLC__1SDV_20170907T102951_20170907T103021_018268_01EB76_Z010.SAFE' #product compressed
ff_safe = 'S1A_IW_SLC__1SDV_20170907T103019_20170907T103047_018268_01EB76_5F55.SAFE' #real product on IRMA eye
path = xsar.get_test_file(ff_safe)
indice_subswath = 2 # from 0 to 2 in IW SLC products
sub_swath_IDs = xsar.Sentinel1Meta(path).subdatasets
sub_swath_IDs

In [ ]:
s1meta = xsar.sentinel1_meta.Sentinel1Meta(sub_swath_IDs[indice_subswath])
s1meta

# what is specific to TOPS SLC products?
Contrarily to GRD products SLC ones preserve the signal described in time per burst. 

It means that a portion of SAR image can be seen up to 4 times by the sensor.

The different bursts are overlapping, and the subswaths are also overlapping.

SLC products also contains the phase information, and digital number in .tiff products are complex values.

## the bursts overlapping areas

In [ ]:
import geoviews as gv
import holoviews as hv
hv.extension('bokeh')
all_polys = []
for indice_subswath in range(3):
    sub_swath_IDs = xsar.Sentinel1Meta(path).subdatasets
    s1meta = xsar.sentinel1_meta.Sentinel1Meta(sub_swath_IDs[indice_subswath])
    onesubswath_burst_polygons  = gv.Path(s1meta.bursts()['geometry'].apply(s1meta.coords2ll)).opts(width=800,height=450)
    all_polys.append(onesubswath_burst_polygons)
gv.tile_sources.Wikipedia * gv.Overlay(all_polys)

# load dataset

In [ ]:
s1ds = xsar.sentinel1_dataset.Sentinel1Dataset(sub_swath_IDs[indice_subswath])
s1ds.dataset

## the azimuth time variable 
the variable `azimuth_time` is given in annotations .xml files, it describes the date of acquisition of each pixel in the subswath. it is also called SAR "long time", in opposition to the "short time" given by the `slant_range_time` variable

In [ ]:

aziHr = s1ds._burst_azitime
hv.Curve(aziHr.values)

`azimuth time` variable estimated at the middle (in range) of the dataset selected is showing the bursts overlapping.

This variable is used to rasterize the 7 variables (longitude, latitude,...) described in the `geolocationGrid` at low resolution.

## the ground range spacing
One specificity of the SLC products is that the ground range spacing is not equal to the slant range spacing (it is the case in GRD products).


In [ ]:
print(s1ds.s1meta.image['ground_pixel_spacing'])
print(s1ds.s1meta.image['slant_pixel_spacing'])


The ground range spacing depends of the incidence angle.

$$ grdRangeSpacing = \frac{slantRangeSpacing}{sinus(\theta)} $$



It is possible for the users to get the ground range spacing vector along the range axis.



In [ ]:
rgs = s1ds.dataset['range_ground_spacing']
rgs

In [ ]:
hv.Curve(rgs).opts(width=400,show_grid=True)


## the complex digital number

In [ ]:
s1ds.dataset['digital_number']

## nice display roughness

In [ ]:
import scipy
import xarray
import os
all_ims = []
prev_atrack = 0
vmax = None
for indice_subswath in range(0,3):
    sub_swath_IDs = xsar.Sentinel1Meta(path).subdatasets
    s1ds = xsar.sentinel1_dataset.Sentinel1Dataset(sub_swath_IDs[indice_subswath],resolution='100m')
    s0 = s1ds.dataset['sigma0'].sel({'pol':'VV'}).real
    print(s0.shape)
    if False:
        s0_averaged = s1ds.dataset['sigma0'].sel({'pol':'VV'}).real.rolling({'atrack':10,'xtrack':10}).mean()
        modulation = s0/s0_averaged
    else:
        modulation = s0
    modulation = xarray.where(np.isnan(modulation),0,modulation)
    modulation = xarray.where(np.isfinite(modulation)==False,0,modulation)
    modulation_detrend = scipy.signal.detrend(modulation, axis=- 1)
    if vmax is None:
        vmax = np.nanpercentile(modulation_detrend,97)
    print('vmax',vmax)
    s1ds.dataset['modulation_detrended'] = xarray.DataArray(modulation_detrend,coords=modulation.coords,dims=modulation.dims)
    s1ds.dataset = s1ds.dataset.assign_coords({'xtrack':s1ds.dataset.xtrack+prev_atrack})
    imi = hv.Image(s1ds.dataset['modulation_detrended'],kdims=['xtrack','atrack']).opts(colorbar=True,
                                                                                        cmap='Greys_r',
                                                                                        width=1200,
                                                                                        height=800,
                                                                                       clim=(0,vmax),
                                                                                        title=os.path.basename(ff_safe))
    prev_atrack += s1ds.s1meta.image['shape'][1]
    all_ims.append(imi)

hv.Overlay(all_ims)

# additional informations

 - deramping TOPS SLC : https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-1-sar/document-library/-/asset_publisher/1dO7RF5fJMbd/content/definition-of-the-tops-slc-deramping-function-for-products-generated-by-the-sentinel-1-ipf;jsessionid=DCEF041CCD5D10A93C637B6121D4D062.jvm1?redirect=https%3A%2F%2Fsentinels.copernicus.eu%2Fweb%2Fsentinel%2Fuser-guides%2Fsentinel-1-sar%2Fdocument-library%3Bjsessionid%3DDCEF041CCD5D10A93C637B6121D4D062.jvm1%3Fp_p_id%3D101_INSTANCE_1dO7RF5fJMbd%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26p_p_col_id%3Dcolumn-1%26p_p_col_count%3D1
 - TOPS technic: https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/topsar-processing
 - SLC products: https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/single-look-complex 